In [ ]:

#  At least 7 machines in the cluster and suggested machine type is n2-highmem-80.



In [1]:
import os
import re
import pandas as pd

from katana import remote
from katana.remote import import_data

# Connect to the Katana Server
client = remote.Client(address="host.docker.internal:8080")

In [2]:
graph = client.create_graph(num_partitions=7)
graph_path = "gs://katana-internal5/Providence-Graphs/Hetio/"
import_data.rdg(graph, graph_path)

          0/? [?op/s]

In [3]:
graph.schema().view()

          0/? [?op/s]

In [4]:
graph = client.create_graph(num_partitions=7)
graph_path = "gs://katana-internal5/Providence-Graphs/Hetio+GDSC/"
import_data.rdg(graph, graph_path)

          0/? [?op/s]

In [5]:
graph.schema().view()

          0/? [?op/s]

In [7]:
graph = client.create_graph(num_partitions=7)
graph_path = "gs://katana-internal48/graph/9JdxR9tyZaSocAbJLWzVAhjWL71YBQbVZNUi3wCJoxyF"
import_data.rdg(graph, graph_path)

          0/? [?op/s]

In [8]:
graph.schema(short_schema=True).view()

          0/? [?op/s]

In [ ]:
%%time
q=""" 

  // optimization - much smaller set of essential genes so start here first before tcga
  MATCH (i:GDSC_IC50_RESULT)-[:HAS_CELL_LINE]->(cl)<-[:HAS_CELL_LINE]-(ach:ACHILLES_ENTRY)-[:HAS_GENE]->(essg:GENE)
  //MATCH (ach:ACHILLES_ENTRY)-[:HAS_GENE]->(essg:GENE)
   where
   i.tcga_cancer_name = 'SKCM'
   and ach.Dependency > .8
   //and essg.symbol = 'PDK1'
  
  //check for common essentials membership
  and essg.isCommonEssential='False'

  // this breaks it
  //and not ((ach)-[:HAS_ENTREZ_GENE]-(x:GENE))

  with distinct essg, i.tcga_cancer_name as cancerType, ach.Dependency as depends limit 1000
 
  
  match (t1:TCGA_GENE)-[o:OBSERVED_MUTATION]->(t2:TCGA_GENE)
  where o.cancer_type = 'SKCM'
  and (toFloat(o.Log2FoldChange )) > 0
  and toFloat(o.MutSigPval) < .2
  and t1.id = essg.symbol  //location SHOULD BE symbol
  and o.cancer_type = cancerType
  //and essg.symbol = 'RRM2'

  // this breaks it because some of genes are not in the tcga set
  //and t2.id  in  ['KRAS','MITF','CDKN2A','JAK2', 'PAX5', 'PPP6C', 'RAC1', 'SNX31', 'TACC1', 'STK19', 'ARID2']

  // confirmed these genes are in the tcga set - will comment in/out as needed
  // commented out right now cause dont find any hits at these threshold levels for just this small set of driver genes
  and t2.id = 'BRAF' //in ['BRAF','KRAS','PPP6C','RAC1','STK19', 'ARID2']
  
  with distinct essg, t2, o.Log2FoldChange as fc, o.MutSigPval as mutsigp ,depends,  cancerType

  match (mutg:GENE)
  where  t2.id = mutg.symbol
  with distinct essg,mutg,cancerType,fc,mutsigp,depends
  
  
 // optional match (essg)-[r]-(mutg)
 //  with mutg,essg, cancerType,count(r) as r1c,fc,mutsigp,depends
   
//  optional match (essg)-[r]-(x:GENE)-[r2]-(mutg)
//   with mutg,essg,r1c,cancerType,count(r2) as r2c,fc,mutsigp,depends
   
//  optional match (essg)-[r]-(x:GENE)-[r2]-(:GENE)-[r3]-(mutg)
//   with mutg,essg,r1c,r2c,cancerType,count(r3) as r3c,fc,mutsigp,depends
  

return *  """

result=graph.query(q,contextualize=True)
result.view()

          0/? [?op/s]

          0/? [?op/s]